---
title: Personalized Prediction Correlations for  868 rn7 genes
date: 9/1/23
author: Sabrina Mi
---

We ran the personalized Enformer pipeline 868 genes for high variation in Br rats and number of eQTLs. There are too many predictions to concatenate iteratively, so I wrote [collect_predictions.py](collect_predictions.py) to parallelize in a submitted job. 

Note: This script appends predictions to the h5 in parallel, I first initialized the h5 file with individuals list to be paired with all gene expression datasets.

In [ ]:
import h5py
import pandas as pd
project_dir = "/home/s1mi/Br_predictions/predictions_folder/personalized_Br_selected_genes"

obs_gene_expr = pd.read_csv("/home/s1mi/enformer_rat_data/expression_data/Brain.rn7.expr.tpm.bed", sep="\t", nrows=1)
with h5py.File(f"{project_dir}/selected_genes_mouse_and_human_predictions.h5", "w") as file:
    file.attrs["index"] = obs_gene_expr.columns.to_list()[4:]

## Read Processed Predictions

In [6]:
import h5py
import pandas as pd
project_dir = "/home/s1mi/Br_predictions/predictions_folder/personalized_Br_selected_genes"
expr_dict = {}
with h5py.File(f"{project_dir}/selected_genes_mouse_and_human_predictions.h5", "r") as file:
    index = file.attrs["index"]
    for gene in file.keys():
        matrix = file[gene][:]
        expr_dict[gene] = pd.DataFrame(matrix, index = index, columns = ["observed", "human predicted", "mouse predicted"])

In [ ]:
corr_by_gene_and_track = pd.DataFrame(columns = ["human", "mouse"], index = gene_list)
for gene in gene_list:
    corr_df = expr_dict[gene].corr()
    corr_by_gene_and_track.loc[gene] = pd.to_numeric(corr_df.iloc[0,1])